<div style="width: 100%; clear: both;">
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M0.536 - Optimización Metaheurística</p>
<p style="margin: 0; text-align:right;">MU Ingeniería Informática / MU Ingeniería Computacional y Matemática</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# _Intro to Optimization and X-Heuristics_

## _Optimization Modeling 101_

### _Reproduce the steps presented in the previous slides_

Pyomo es un paquete que provee de un conjunto diverso de capacidades de optimización para formular, resolver y analizar modelos de optimización.

Para este ejercicio se ha utilizado el modelo diet.dat que proporciona la misma librería. En dicha dieta podemos encontrar asegura una dieta completa minimizando el coste.

Antes de modelar es importante mencionar que Pyomo no provee de _solvers_ por defecto y se tienen que instalar aparte. Para este ejercicio se ha utilizado el solver [GLPK](https://www.gnu.org/software/glpk/) (_GNU Linear Programming Kit_),  diseñado para resolver problemas de programación lineal (LP) a gran escala, programación entera mixta (MIP) y otros problemas relacionados.

El modelado se hará en el archivo diet.py donde tendremos

* **Conjuntos**: Comida y Nutrientes
* **Parámetros**: Coste de ración, cantidad de nutrientes por comida, baremos de nutrientes
* **Variables**: Ración
* **Ojetivo**: minimizar el coste de la comida
* **Restricciones**: tiene que haber un mínimo de nutriente por comida y ración

Cada uno de estos elementos de nuestro modelo tiene, a su vez, un dominio: Reales positivos o Reales no negativos).

In [1]:
from pyomo.environ import *


model = AbstractModel()

model.FOOD = Set()
model.cost = Param(model.FOOD, within=PositiveReals)

model.f_min = Param(model.FOOD, within=NonNegativeReals, default=0.0)

def f_max_validate (model, food):
    return model.f_max[food] > model.f_min[food]
model.f_max = Param(model.FOOD, validate=f_max_validate, default=20.0)

model.NUTR = Set()
model.n_min = Param(model.NUTR, within=NonNegativeReals, default=0.0)
model.n_max = Param(model.NUTR, default=float("inf"))
model.amt = Param(model.NUTR, model.FOOD, within=NonNegativeReals)

model.servings = Var(model.FOOD, within=NonNegativeIntegers)

def total_cost_rule(model):
    return sum(model.cost[food] * model.servings[food] for food in model.FOOD)
model.total_cost = Objective(rule=total_cost_rule, sense=minimize)


def nutrient_rule(model, nutrient):
    limit = sum(model.amt[nutrient, food] * model.servings[food] for food in model.FOOD)
    return (model.n_min[nutrient], limit, model.n_max[nutrient])
model.nutrient_limit = Constraint(model.NUTR, rule=nutrient_rule)

La ejecución del comando de Pyomo generará un archivo results.yml

In [3]:
!pyomo solve --solver=glpk diet.py diet.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    1.53] Applying solver
[    1.53] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 15.05
    Solver results file: results.yml
[    1.54] Applying Pyomo postprocessing actions
[    1.54] Pyomo Finished


In [4]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 15.05
  Upper bound: 15.05
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 10
  Number of nonzeros: 68
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 41
      Number of created subproblems: 41
  Error rc: 0
  Time: 0.002698659896850586
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

Como resultado, Pyomo junto con GLPK ha encontrado una solución para nuestra dieta que consiste en una ración de patatas pequeñas, Filet-O-Fish, Leche desnatada 1% y un cuarto de libra con queso.

### _Choose one tool and test it with an IP/MIP example from its website_

Se puede utilizar Pyomo para resolver problemas IP de manera sencilla. Dado el siguiente modelo:

$$ \textrm{min} \quad  2x_{1} + 3x_{2} $$
$$ \textrm{s.t.} \quad  3x_{1} + 4x_{2} \geqslant 1$$
$$ x_{1},x_{2} \geqslant 0 $$

Se puede implementar con Pyomo de la siguiente forma:

In [21]:
from pyomo.environ import *

model = ConcreteModel()
model.x = Var([1,2], domain=NonNegativeReals)
model.OBJ = Objective(expr = 2*model.x[1] + 3*model.x[2])
model.Constraint1 = Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)

### _Choose one tool and test it with an NLP example from its website_

PyNumero es un paquete para desarrollar algoritmos paralelos para programas no lineales (NLP) en Pyomo. Para este ejercicio se añadirá una dependencia más llamada numpy, paquete de matemáticas para Python.  
A continuación paso a describir el uso de PyNumero para la evaluación de funciones y derivadas.

En primer lugar creamos un modelo y creamos una instancia de PyomoNLP

In [5]:
import pyomo.environ as pe
from pyomo.contrib.pynumero.interfaces.pyomo_nlp import PyomoNLP

m = pe.ConcreteModel()
m.x = pe.Var(bounds=(-5, None))
m.y = pe.Var(initialize=2.5)
m.obj = pe.Objective(expr=m.x**2 + m.y**2)
m.c1 = pe.Constraint(expr=m.y == (m.x - 1)**2)
m.c2 = pe.Constraint(expr=m.y >= pe.exp(m.x))


nlp = PyomoNLP(m)

Una vez creado el modelo, podemos realizar distintas operaciones con PyNumero como los límites y restricciones de las variables

In [10]:
nlp.primals_lb(), nlp.primals_ub(), nlp.constraints_lb(), nlp.constraints_ub()

(array([ -5., -inf]), array([inf, inf]), array([  0., -inf]), array([0., 0.]))

El objetivo y las restricciones del modelo

In [11]:
nlp.evaluate_objective(), nlp.evaluate_constraints()

(6.25, array([ 1.5, -1.5]))

Evaluaciones diversas

In [18]:
jacobian = nlp.evaluate_jacobian()
print(jacobian)
jacobian

  (0, 0)	2.0
  (1, 0)	1.0
  (0, 1)	1.0
  (1, 1)	-1.0


<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in COOrdinate format>

In [20]:
hessian_lag = nlp.evaluate_hessian_lag()
print(hessian_lag)
hessian_lag

  (0, 0)	2.0
  (1, 1)	2.0


<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in COOrdinate format>

Podemos establecer valores

In [23]:
from numpy import array

nlp.set_primals(array([0, 1]))
nlp.set_duals(array([-2/3, 4/3]))
nlp.evaluate_grad_objective() + nlp.evaluate_jacobian().transpose() * nlp.get_duals()

array([0., 0.])

Y, finalmente, igualdades e inigualdades

In [24]:
nlp.evaluate_eq_constraints(), nlp.evaluate_ineq_constraints()

(array([0.]), array([0.]))

In [25]:
nlp.get_duals_eq(), nlp.get_duals_ineq()

(array([-0.66666667]), array([1.33333333]))

## _Intro to x-Heuristic Optimization_

### _Watch the following video on the concept of Simheuristics and write a short summary on it_

La complejidad de nuestras interacciones se ha visto incrementada en los últimos años, no sólo por las interacciones en sí sino por las nuevas capacidades y variedad de los sistemas con los que interactuamos: transporte, compras, IoT, etc. Dichas interacciones suponen un coste económica y suelen acarrear problemas medioambientales. Para mitigar estos problemas surgen algoritmos informáticos que nos permiten analizar esta información e intentar identificar patrones para optimizar la eficiencia, a este conjunto de técnicas y algoritmos se le denomina metaheurística.  
Sin embargo, las interacciones suelen tener elementos no contemplados e incontrolables. Para poder modelar y reproducir estos sistemas e intentar predecir estos sucesos aleatorios usamos las simulaciones, pero estas, a su vez, están limitadas y deben integrarse con análisis y metaheurísticas dando como resultado la simheurística. La simheurística es una metodología con muchas capacidades, flexible y fácil de implementar dando soluciones adaptadas a los problemas anteriormente mencionados.

### _Read the following papers and prepare a summary about them_

* **_Electric Vehicles in Logistics and Transportation: A Survey on Emerging Environmental, Strategic, and Operational Challenges_**

Este paper identifica las distintas líneas de investigación y retos relaciones con la introducción de vehículos eléctricos en la logística y transporte con flotas heterogéneas, tales como los problemas mediambientales, planificación estratégica y otros problemas asociados a los vehículos eléctricos o basados en hidrógeno. Además, propone variantes al conocido problema de _Vehicle Routing Problem_ (VRP) mediante metaheurísticas y simheurísticas.

Los sistemas de logística y transporte se han estudiado ampliamente en el campo de investigación operativa durante mucho tiempo para optimizar la asignación de vehículos, es lo que se conoce como el problema del _Vehicle Routing Problem_. Una conocida variante de este problema es el _Capacitated VRP (CVRP)_ donde cada usuario tiene una demanda a satisfacer sin exceder la capacidad del vehículo; también existe la variante de _VRP_ con ventanas de tiempo en donde existen márgenes temporales para almacén y otro para usuarios, se habla también de _Multi-attribute VRP_ o _Rich VRP_.  
Debido a que el sector del transporte se estima que produce el 28% del total de los gases de efecto invernadero en países como Estados Unidos las emisiones se incorpora como un objetivo a minimizar, sin embargo, otro enfoque es el uso de vehículos menos contaminantes como los vehículos eléctricos. Existen varias líneas de investigación abiertas relacionadas con estos vehículos y actividades de logística:

* Problemas medioambientales
* Problemas de planificación estratégica asociados con los vehículos eléctricos
* Problemas operacionales emergentes relacionados con el uso de vehículos eléctricos en _VRPs_

Respecto al problema medioambiental se concretan distintos problemas que conlleva el transporte y la logística pero se pone énfasis en 2: contaminación acústica y polución. La importante de controlar los distintos tipos de emisiones contaminantes justifica la búsqueda de nuevas tecnologías que nos ayuden a reducir el impacto ambiental de fragatas de transportes.  

Los problemas relacionados con el abastacemiento de ciudades son crísticos para la logística y el transporte ya que el alrededor del 80% de la población europea y estado unidense vive en zonas urbanas y existen estudios que contabilizan que entre el 6% y 18% del total de recorrido urbano, el 19% de la energía consumida y el 21% de la polución corresponden a carga urbana. El cambio a vehículos eléctricos puede suponer una buena solución para paliar problemas relacionados con la distribución urbana y minimizar el consumo de combustible incorporando restricciones de batería a estrategías de gestión de energía. También se propone un modelo estocástico para minimizar el coste total de recorrido camiones que incluyen tiempos de entrega, diferentes tipos de emisiones y penalización por llegar tarde o demasiado pronto.  
Existen muchas publicaciones comentando las propiedades y ventajas de los vehículos eléctricos pero muy poco en el impacto ambiental de la producción de los mismos y la generación de electricidad que eso conlleva.  
El proceso de distribución normalmente se vuelve crítico en el último tramo y es donde las decisiones más difíciles de operaciones tienen que ocurrir. Los vehículos eléctricos han demostrado ser muy útiles para descarbonizar este último tramo. Haciendo una comparación directa entre las emisiones de estos vehículos con los de combustión interna vemos que las emisiones caen un 20%.   

La inclusión de emisiones en problemas de planificación de rutas ha dado paso a nuevos modelos y al desarrollo de nuevos algoritmos. Dado que muchos aspectos difieren entre vehículos de combustión y los eléctricos hay que reformular los problemas asociados al transporte con vehículos:

* Número y tipo de estaciones para recargar vehículos eléctricos
* Localización de las estaciones
* Capacidad óptima

Debido a la limitación de la duración de la batería el establecimiento de una infraestructura distribuida para el cambio de batería podría ser importante para establecer nuevas políticas de transporte. Los grandes retos de los puntos de recarga tiene que ver con el tipo de estas: de recarga lenta que ralentiza el tiempo de uso del vehículo y los de recarga rápida que erosiona la batería, es por esto que la estrategia del cambio de batería ha cobrado fuerza estos últimos años.  
Otro problema a considerar es la localización de estos puntos de recarga que, a su vez, es un problema de _Facility Location Problem (FCP)_:

* Número de instalaciones
* Localización de estas
* Tipo de instalación

Para solventar el problema del número de instalaciones en viajes de larga distancia se propone el modelo de _Flow Refueling Location Problem (FRLM)_ para maximizar el flujo del tráfico. Sin embargo, estos modelos no consideran el tiempo de carga o eficiencia y se limitan a un solo tipo de estación de recarga.  
Otros modelos son propuestos para considerar tipos de viajes, combinación de localizaciones y tipos de puntos de carga, etc.

En cuanto a la capacidad de estos puntos de recarga solo un número reducido de coches es capaz de recargar y se producen colas. Se proponen modelos que permitan preveer la demanda de recarga en ciertos puntos concretos de zonas urbanas. También se proponen estaciones de cambio de batería y puntos de distribución.

Existen distintas variables, restricciones y objetivos que surgen cuando se enfoca el _VRP_ a los vehículos eléctricos, concretamente surgen variantes relacionadas con el tamaño y propiedades de la flota de vehículos, redes de recarga y enrutamiento. Esto hace que los emergentes vehículos supongan un reto operacional que debe ser debidamente dirigido.

**_Puntuación_**  

Opino que es un paper muy completo y que se podría entender como una especie de survey que expone el estado del arte de la investigación operativa enfocada en un sector muy concreto. La presentación y separación de secciones me ha parecido muy coherente y fácil de seguir, sin embargo, creo que en algunos puntos reitera demasiadas veces las mismas ideas y no aterriza tanto al detalle de cómo estos problemas pueden ser solventados.

Resultado: 8/10

----------

* **_The Team Orienteering Problem With Stochastic Service Times And Driving-Range Limitations: A Simheuristic Approach_**

Este paper hablar de una versión muy concreta del problema del _Team Orienteering Problem (TOP)_ donde una flota de vehículos no manejados (UAV) tiene que realizar una serie de paradas o visitar a clientes. Se asume que la cantidad de recompensa recibida es una variable aleatoria, el objetivo es llegar a un conjunto óptimo de clientes y la restricción es que cada UAV vuele dentro del rango definido.


Este paper estudia una versión estocástica con recompensas del del _TOP_. Se propone un algoritmo simheurístico de recompensas con tiempos de servico y duración de rutas. Combina una simulación Monte Carlo con un marco de trabajo metaheurístico y técnicas randomización sesgada (_biased randomization_).  

La descripción del problema se puede definir como un grafo no dirigido con nodos y aristas. Se define una ruta que empiza en un punto de origen y termina en un punto de destino. Cada cliente que pertenece al conjunto de nodos tiene una recompensa estocástica y un tiempo de servicio. El tiempo de servicio sigue una ecuación con una relación entre el tiempo de servicio y la recompensa. El tiempo total del viaje vendrá determinado por la batería de cada UAV lo cual establece una restricción en el algoritmo que no puede ser garantizada con tiempos aleatorios por tanto, en la práctica, pueden ocurrir fallos en las rutas.  
El objetivo es determinar el subconjunto de usuarios que será incluído en cada ruta. Si la ruta no es completada se perderá la recompensa y los clientes con más recompensa requieren más tiempo de servicio.

Los algoritmos simheurísticos tienen 2 componentes: uno de optimización que busca soluciones prometedoras y otro de simulación que prueba estas soluciones en un entorno estocástico y guía el proceso de búsqueda. El algoritmo se compone, a su vez de 3 fases:

1. Fase inicial o "_dummy_" construyendo una ruta conectando a los usuarios. En este nivel se introduce el concepto de "_savings_" o ahorro que es el tiempo ahorrado cuando 2 rutas se juntan. A su vez, nace el concepto de "_preference_" o preferencia que es utilizado para generar una lista de combinaciones de ruta potenciales.
2. En la segunda fase se utilizan técnicas de randomización sesgadas para transformar la heurística anterior en un algoritmo probabilístico
3. En este punto se evalúa la calidad de la solución generada. Una solución se define para la versión determinista y la estocástica. El criterio de aceptación decide si la nueva solución es prometedora o no, si no lo es entonces otra iteración comienza.
4. Finalmente una simulación más intensa tiene lugar con cada una de las soluciones preseleccionadas

Se asume que las recompensas estocásticas siguen una distribución normal.

Para validar la calidad de este enfoque se compara los resultados con la mejor solución conocida para cada instancia. La mejor solución estocástica provee una solución que es, de media, un 4.17% mejor que la recompensa esperada por la solución determinística. Como es esperado, la solución media incrementa con el tamaño de la flota.

**_Puntuación_**

Opino que se trata de un paper muy bien explicado, entra al detalle de una manera muy clara y directa al problema y con pocos conocimientos de matemáticas se puede comprender lo que se está explicando. Creo además que es un paper que abarca temas modernos e importantes como son los drones y smart cities, además, resulta que trabajo en este sector y me ha resultado muy agradable y didáctico.

Resultado: 10/10

# _Random Search and VRPs_

## _Construct your own Python program to implement a RS algorithm for solving the BFP._

In [2]:
from time import time


# helper function
def timeit(func):
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s')
        return result
    return wrap_func

In [56]:
from random import random


def basin(_list):
    n, a, h, k = 2, 0.5, 2, -5
    return sum([a * pow((element - h), n) + k for element in _list])


def random_solution(space, size):
    minimum, maximum = space
    return [minimum + (maximum - minimum) * random() for _ in range(0, size)]


@timeit
def random_search(space, size, iterations):
    best_solution, best_cost = None, float("inf")
    while iterations > 0:
        iterations -= 1
        solution = random_solution(space, size)
        cost = basin(solution)
        if cost < best_cost:
            best_solution, best_cost = solution, cost
    return best_solution, best_cost
            

space = [-5, 5]
sizes = [2, 4, 8]
iterations_col = [10000, 20000]


for size in sizes:
    for iterations in iterations_col:
        solution, cost = random_search(space, size, iterations)
        print(f"Size: {size}; Iterations: {iterations};")
        print(f"Solution: {solution}\nCost: {cost}\n")

Function 'random_search' executed in 0.0248s
Size: 2; Iterations: 10000;
Solution: [2.0222333008092575, 1.9412748325333737]
Cost: -9.99802851752057

Function 'random_search' executed in 0.0381s
Size: 2; Iterations: 20000;
Solution: [2.020641341973649, 2.0095270819731326]
Cost: -9.999741584855302

Function 'random_search' executed in 0.0362s
Size: 4; Iterations: 10000;
Solution: [1.7252061406200712, 2.274898153524986, 1.7733564128486181, 1.9410202066223405]
Cost: -19.89703670420596

Function 'random_search' executed in 0.0440s
Size: 4; Iterations: 20000;
Solution: [2.154474693697006, 1.816437154459817, 2.2352387550008332, 2.1340155077746275]
Cost: -19.934572411282986

Function 'random_search' executed in 0.0315s
Size: 8; Iterations: 10000;
Solution: [2.853941276259164, 3.208009752069037, 2.2521425719129784, 2.91688041027487, 1.8171263958241957, 2.1266283716282075, 2.3692040518795725, 2.048906542854785]
Cost: -38.35953509541325

Function 'random_search' executed in 0.0632s
Size: 8; Itera

## _Solve the same problem but for the following basin function_

$$ f(x) = \sum_{i=1}^n x_{i}^2 $$
$$ \forall -5.0 < x_{i} < 5.0 \quad \textrm{and} \quad  n = 2 $$

In [58]:
def basin(_list):
    n = 2
    return sum([pow(element, n) for element in _list])


for size in sizes:
    for iterations in iterations_col:
        solution, cost = random_search(space, size, iterations)
        print(f"Size: {size}; Iterations: {iterations};")
        print(f"Solution: {solution}\nCost: {cost}\n")

Function 'random_search' executed in 0.0311s
Size: 2; Iterations: 10000;
Solution: [0.020784068043769643, -0.02270938930178623]
Cost: 0.0009476938469081294

Function 'random_search' executed in 0.0397s
Size: 2; Iterations: 20000;
Solution: [-0.026773757308037105, -0.02663026957162895]
Cost: 0.0014260053378472972

Function 'random_search' executed in 0.0193s
Size: 4; Iterations: 10000;
Solution: [0.1798137167220002, -0.6790354553047422, 0.012190912321483616, -0.16030953642784862]
Cost: 0.5192698880952401

Function 'random_search' executed in 0.0360s
Size: 4; Iterations: 20000;
Solution: [-0.3110557058733381, 0.14200236047924442, 0.03463736403980544, 0.48648397110933317]
Cost: 0.3547867236719704

Function 'random_search' executed in 0.0267s
Size: 8; Iterations: 10000;
Solution: [0.6014664970050863, 0.2077590493925907, 0.28315530616973117, -0.8095924429439272, -1.323547442209044, 1.6075093030941456, 0.3439021167173255, -1.7212500679077003]
Cost: 8.55737707417532

Function 'random_search' 

## _Construct your own Python program to implement the CWS heuristic for solving the VRP and test it in different instances_

Este ejercicio se compone de 2 elementos de código, el primero es la definición de las clases Nodo, Arista, Ruta y Solución que será reutilizado en el ejercicio 4.

El segundo componente es un programa para resolver VRP siguiendo las instrucciones de los apuntes, sin emabargo y lamentablemente debe haber algún error en la codificación y no funciona.

In [17]:
class Node:
    def __init__(self, ID, x, y, demand):
        self.ID = ID
        self.x = x
        self.y = y
        self.demand = demand
        self.in_route = None
        self.interior = False
        self.dn_edge = None
        self.nd_edge = None
        self.start_linked = False
        self.finish_linked = False


class Edge:
    def __init__(self, origin, end):
        self.origin = origin
        self.end = end
        self.cost = 0.0
        self.savings = 0.0
        self.inverted_edge = None
        self.efficiency = 0.0


class Route:
    def __init__(self):
        self.cost = 0.0
        self.edges = []
        self.demand = 0.0
    
    def reverse(self):
        size = len(self.edges)
        for i in range(size):
            edge = self.edges[i]
            inverted_edge = edge.inverted_edge
            self.edges.remove(edge)
            self.edges.insert(0, inverted_edge)

            
class Solution:
    last_ID = -1

    def __init__(self):
        Solution.last_ID += 1
        self.ID = Solution.last_ID
        self.routes = []
        self.cost = 0.0
        self.demand = 0.0

In [ ]:
import math
import operator


def read_nodes(filename):
    nodes = []
    with open(filename) as instance:
        for i, line in enumerate(instance):
            node_data = [float(x) for x in line.split()]
            nodes.append(Node(i, node_data[0], node_data[1], node_data[2]))
    return nodes


def get_depot_edge(a_route, a_node, depot):
    origin = a_route.edges[0].origin
    end = a_route.edges[0].end
    if ((origin == a_node and end == depot) or
        (origin == depot and end == a_node)):
        return a_route.edges[0]
    else:
        return a_route.edges[-1]


def vrp(alpha, nodes, veh_cap):    
    def is_mergeable(i_node, j_node, i_route, j_route, ij_edge):
        if i_route == j_route: return False
        if i_node.interior or j_node.interior: return False
        if veh_cap < i_route.demand + j_route.demand: return False
        return True

    
    depot = nodes[0]

    for node in nodes[1:]:
        dn_edge = Edge(depot, node)
        nd_edge = Edge(node, depot)
        dn_edge.inverted_edge = nd_edge
        nd_edge.inverted_edge = dn_edge
        dn_edge.cost = math.sqrt((node.x - depot.x)**2 + (node.y - depot.y)**2)
        nd_edge.cost = dn_edge.cost
        node.dn_edge = dn_edge
        node.nd_edge = nd_edge

    savings = []
    for i in range(1, len(nodes) - 1):
        i_node = nodes[i]
        for j in range(i+1, len(nodes)):
            j_node = nodes[j]
            ij_edge = Edge(i_node, j_node)
            ji_edge = Edge(j_node, i_node)
            ij_edge.inverted_edge = ji_edge
            ji_edge.inverted_edge = ij_edge
            ij_edge.cost = math.sqrt((j_node.x - i_node.x)**2) + (j_node.y - i_node.y)**2
            ji_edge.cost = ij_edge.cost
            ij_edge.savings = i_node.nd_edge.cost + j_node.dn_edge.cost - ij_edge.cost
            ji_edge.savings = ij_edge.savings
            savings += [ij_edge]

    savings.sort(key=operator.attrgetter("savings"), reverse=True)

    # construct solution
    solution = Solution()
    for node in nodes[1:-1]:
        dn_edge = node.dn_edge
        nd_edge = node.nd_edge
        dnd_route = Route()
        dnd_route.edges.append(dn_edge)
        dnd_route.demand += node.demand
        dnd_route.cost += dn_edge.cost
        dnd_route.edges.append(nd_edge)
        dnd_route.cost += nd_edge.cost
        node.in_route = dnd_route
        node.interior = False
        solution.routes.append(dnd_route)
        solution.cost += dnd_route.cost
        solution.demand += dnd_route.demand

    while len(savings) > 0:
        ij_edge = savings.pop(0)
        i_node = ij_edge.origin
        j_node = ij_edge.end
        i_route = i_node.in_route
        j_route = j_node.in_route
        if is_mergeable(i_node, j_node, i_route, j_route, ij_edge):
            i_edge = get_depot_edge(i_route, i_node, depot)
            i_route.edges.remove(i_edge)
            i_route.cost -= i_edge.cost
            if len(i_route.edges) > 1:
                i_node.interior = True
            if i_route.edges[0].origin != depot:
                i_route.reverse()
            j_edge = get_depot_edge(j_route, j_node, depot)
            j_route.edges.remove(j_edge)
            j_route.cost -= j_edge.cost
            if len(j_route.edges) > 1:
                j_node.interior = True
            if j_route.edges[0].origin != depot:
                j_route.reverse()
            j_route.edges.append(ij_edge)
            i_route.cost += ij_edge.cost
            j_route.demand += j_node.demand
            j_node.in_route = i_route

            for edge in j_route.edges:
                i_route.edges.append(edge)
                i_route.cost += edge.cost
                i_route.demand += edge.end.demand
                edge.end.in_route = i_route
            solution.cost -= ij_edge.savings
            solution.routes.remove(j_route)

    return solution


alphas = [0.3, 0.5, 1.0]
files = ["A-n80-k10_input_nodes.txt"]
veh_cap = 100.0

for alpha in alphas:
    for _file in files:
        instance_name = _file.split(".txt")[0]
        nodes = read_nodes(_file)
        solution = vrp(alpha, nodes, veh_cap)
        print(f"alpha: {alpha} || Veh Cap: {veh_cap}")
        for route in solution.routes:
            s = str(0)
            for edge in route.edges:
                s = s + '-' + str(edge.end.ID)
            print(f"Route: {s} || Cost = {route.demand:.2f}")


# _Biased-Randomized Algorithms (BRAs)_

## _Explain in detail the Python code discussed in class, and how it relates to the biased randomization concept._

In [19]:
from math import log
from random import random


efficiency_list = [2, 1, 3, 4, 6, 5, 7, 9, 0, 8]
efficiency_list.sort(reverse=True)

# parameter of geometric probability distribution
alphas = [0.0001, 0.3, 0.6, 0.9999]

for alpha in alphas:
    print(f"alpha {alpha}")
    for i in range(10):
        eff_list = efficiency_list.copy()
        store_list = []
        for _ in range(len(eff_list)):
            # generate random number from geometric distribution
            index = int(log(random())/log(1-alpha))
            index = index % len(eff_list)
            store_list.append(eff_list[index])
            eff_list.pop(index)
        
        print(f"Biased-Rand {i}: {store_list}")
    print("")


alpha 0.0001
Biased-Rand 0: [3, 8, 2, 0, 6, 4, 7, 9, 5, 1]
Biased-Rand 1: [7, 0, 3, 6, 4, 8, 2, 9, 1, 5]
Biased-Rand 2: [3, 4, 7, 6, 1, 0, 5, 9, 8, 2]
Biased-Rand 3: [1, 3, 7, 2, 6, 0, 5, 9, 8, 4]
Biased-Rand 4: [3, 5, 6, 8, 1, 2, 7, 9, 0, 4]
Biased-Rand 5: [8, 4, 3, 0, 9, 1, 7, 5, 2, 6]
Biased-Rand 6: [7, 5, 1, 3, 8, 0, 9, 4, 6, 2]
Biased-Rand 7: [2, 3, 5, 0, 9, 1, 7, 8, 4, 6]
Biased-Rand 8: [0, 8, 4, 7, 2, 9, 3, 6, 1, 5]
Biased-Rand 9: [8, 7, 9, 2, 5, 0, 3, 6, 1, 4]

alpha 0.3
Biased-Rand 0: [3, 9, 6, 7, 4, 8, 5, 2, 0, 1]
Biased-Rand 1: [5, 9, 8, 2, 4, 6, 3, 1, 7, 0]
Biased-Rand 2: [8, 3, 7, 0, 9, 4, 5, 1, 6, 2]
Biased-Rand 3: [9, 8, 3, 5, 6, 4, 1, 7, 2, 0]
Biased-Rand 4: [6, 7, 5, 2, 9, 3, 0, 4, 8, 1]
Biased-Rand 5: [6, 8, 2, 9, 0, 4, 7, 1, 3, 5]
Biased-Rand 6: [9, 8, 6, 4, 5, 3, 7, 1, 2, 0]
Biased-Rand 7: [9, 8, 7, 5, 6, 3, 4, 1, 2, 0]
Biased-Rand 8: [9, 3, 2, 8, 4, 7, 5, 1, 0, 6]
Biased-Rand 9: [7, 8, 3, 6, 9, 4, 1, 5, 2, 0]

alpha 0.6
Biased-Rand 0: [9, 7, 8, 5, 3, 6, 2, 4, 1, 0]

Se declara una lista que será ordenada siguiendo un criterio heurístico, en este caso, en sentido opuesto.  

Se copia la lista y se itera una serie de veces. En cada iteración, por cada elemento de la lista se seleccionará un índice siguiendo una distribución geométrica que vendrá determinado por un parámetro `alpha`. Este índice truncado por el tamaño de la lista nos dará un elemento al azar que insertaremos en la segunda lista.

Como podemos observar, cuanto más alto es la variable `alpha` más se acerca al comportamiento "_greedy_": Elementos ordenados de mayor a menor, es decir, hay más probabilidad de seleccionar el primer elemento. Cuanto más cerca a 0 el algoritmo se comporta como una distribución aleatoria y genera una lista con los elementos más desordenados. He aquí el sesgo.

## _Search in Google Scholar for papers containing the words “biased randomization”. Generate a table by application area._

Para este ejercicio se han seleccionado 10 artículos de Google Scholar con la keyword de "biased randomization"

| Artículo  | Application Area |
|---|---|
| [Biased randomization of heuristics using skewed probability distributions: A survey and some applications](https://www.sciencedirect.com/science/article/abs/pii/S036083521730270X)  | Computers & Industrial Engineering  |
|  [Enhancing Stochastic Search Performance by Value-Biased Randomization of Heuristics](https://link.springer.com/article/10.1007/s10732-005-6997-8) | Combinatorics  |
| [An ILS-biased randomization algorithm for the two-dimensional loading HFVRP with sequential loading and items rotation](https://www.tandfonline.com/doi/abs/10.1057/jors.2015.48)  | Operational Research  |
| [Using biased randomization for solving the two-dimensional loading vehicle routing problem with heterogeneous fleet](https://link.springer.com/article/10.1007/s10479-014-1551-4)  | Vehicle Routing Problem  |
| [MIRHA: multi-start biased randomization of heuristics with adaptive local search for solving non-smooth routing problems](https://link.springer.com/article/10.1007/s11750-011-0245-1)  | Vehicle Routing Problem  |
| [High-throughput evaluation of T7 promoter variants using biased randomization and DNA barcoding](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0196905)  | Medicine  |
| [A design for testing clinical strategies: biased adaptive within-subject randomization](https://rss.onlinelibrary.wiley.com/doi/abs/10.1111/1467-985X.00154)  | Medicine  |
| [Avoiding bias from weak instruments in Mendelian randomization studies](https://academic.oup.com/ije/article/40/3/755/745918)  | Genetics  |
| [A Method of Biased Coin Randomization, Its Implementation, and Its Validation](https://link.springer.com/article/10.1177/009286159803200213)  | Mathematics |

# _GRASP & PJS Heuristic TOP_

## _Construct your own Python program to implement a GRASP algorithm for solving the TSP._

Se ha implementado el algoritmo de GRASP para resolver TSP con diferentes alfas (_greed factors_) y diferentes iteraciones sobre un TSP fijo.

In [3]:
from math import sqrt
from random import randrange
from time import time


def distance(node_1, node_2):
    return sqrt((node_1[0] - node_2[0]) ** 2.0 + (node_1[1] - node_2[1]) ** 2)


def tour_cost(solution):
    total = 0.0
    for i, node in enumerate(solution):
        end_node = solution[0] if i == len(solution) - 1 else solution[i + 1]
        total += distance(node, end_node)
    return total


def stochastic_two_opt(solution):
    permutation = solution[:]
    size = len(permutation)
    c1, c2 = randrange(0, size), randrange(0, size)
    exclude = {c1}
    exclude.add(size - 1 if c1 == 0 else c1 - 1)
    exclude.add(0 if c1 == size - 1 else c1 + 1)

    while c2 in exclude:
        c2 = randrange(0, size)

    if c2 < c1:
        c1, c2 = c2, c1
    permutation[c1:c2] = reversed(permutation[c1:c2])
    return permutation


def local_search(solution, cost, max_iterations):
    counter = 0
    while counter < max_iterations:
        new_solution = stochastic_two_opt(solution)
        new_cost = tour_cost(new_solution)
        if new_cost < cost:
            solution, cost = new_solution, new_cost
        else:
            counter += 1
    return solution, cost


def greedy(solution, greed_factor):
    size = len(solution)
    candidate = [solution[randrange(0, size)]]
    while len(candidate) < size:
        candidates = [node for node in solution if node not in candidate]
        costs = [distance(candidate[len(candidate) - 1], node) for node in candidates]
        min_cost, max_cost = min(costs), max(costs)
        rcl = [
            candidates[i]
            for i, cost in enumerate(costs)
            if cost <= min_cost + greed_factor * (max_cost - min_cost)
        ]
        candidate.append(rcl[randrange(0, len(rcl))])
    return candidate, tour_cost(candidate)

@timeit
def grasp(input_tsp, greed_factor, max_iterations=1000, max_improves=50):
    best_cost, best_solution = float("inf"), None
    iterations = max_iterations

    while iterations > 0:
        iterations -= 1
        solution, cost = greedy(input_tsp, greed_factor)
        solution, cost = local_search(solution, cost, max_improves)
        if cost < best_cost:
            print(f"New best cost {best_cost:.2f} at iteration {max_iterations - iterations}")
            best_solution, best_cost = solution, cost
    return best_solution, best_cost

# input TSP will be 52 locations of Berlin
berlin_52 = [
    [565, 575], [25, 185], [345, 750], [945, 685], [845, 655], [880, 660], [25, 230], [525, 1000], [580, 1175],
    [650, 1130], [1605, 620], [1220, 580], [1465, 200], [1530, 5], [845, 680], [725, 370], [145, 665], [415, 635],
    [510, 875], [560, 365], [300, 465], [520, 585], [480, 415], [835, 625], [975, 580], [1215, 245], [1320, 315],
    [1250, 400], [660, 180], [410, 250], [420, 555], [575, 665], [1150, 1160], [700, 580], [685, 595], [685, 610],
    [770, 610], [795, 645], [720, 635], [760, 650], [475, 960], [95, 260], [875, 920], [700, 500], [555, 815],
    [830, 485],  [1170, 65], [830, 610], [605, 625], [595, 360], [1340, 725], [1740, 245],
]

greed_factors = [0.2, 0.3, 0.5, 0.7, 0.9]
iterations = [1000, 2000, 5000]
for greed_factor in greed_factors:
    for iteration in iterations:
        print(f"Greed factor of {greed_factor}. Number of iterations is {iteration}")
        result = grasp(berlin_52, greed_factor, iteration)
        print(f"{result}\n")

Greed factor of 0.2. Number of iterations is 1000
New best cost inf at iteration 1
New best cost 12475.07 at iteration 13
New best cost 11577.30 at iteration 156
New best cost 10758.36 at iteration 203
Function 'grasp' executed in 4.3290s
([[1220, 580], [1250, 400], [1215, 245], [1170, 65], [1320, 315], [1465, 200], [1530, 5], [1740, 245], [1605, 620], [1340, 725], [830, 610], [830, 485], [700, 580], [700, 500], [685, 595], [880, 660], [795, 645], [725, 370], [660, 180], [410, 250], [595, 360], [560, 365], [145, 665], [345, 750], [525, 1000], [475, 960], [510, 875], [555, 815], [520, 585], [605, 625], [415, 635], [420, 555], [25, 185], [25, 230], [95, 260], [300, 465], [480, 415], [565, 575], [575, 665], [685, 610], [720, 635], [835, 625], [760, 650], [770, 610], [845, 655], [975, 580], [845, 680], [945, 685], [875, 920], [650, 1130], [580, 1175], [1150, 1160]], 10453.495332201228)

Greed factor of 0.2. Number of iterations is 2000
New best cost inf at iteration 1
New best cost 13665.5

New best cost 16205.89 at iteration 116
Function 'grasp' executed in 22.1998s
([[880, 660], [830, 610], [605, 625], [575, 665], [835, 625], [975, 580], [845, 655], [875, 920], [650, 1130], [525, 1000], [510, 875], [475, 960], [420, 555], [145, 665], [345, 750], [480, 415], [410, 250], [25, 185], [95, 260], [25, 230], [415, 635], [595, 360], [555, 815], [520, 585], [760, 650], [720, 635], [700, 580], [795, 645], [580, 1175], [1150, 1160], [945, 685], [1220, 580], [770, 610], [845, 680], [565, 575], [300, 465], [560, 365], [725, 370], [700, 500], [830, 485], [660, 180], [685, 610], [685, 595], [1250, 400], [1215, 245], [1465, 200], [1170, 65], [1320, 315], [1530, 5], [1740, 245], [1605, 620], [1340, 725]], 14019.498500898177)

Greed factor of 0.7. Number of iterations is 1000
New best cost inf at iteration 1
New best cost 21471.36 at iteration 2
New best cost 20847.33 at iteration 6
New best cost 20409.78 at iteration 7
New best cost 20094.39 at iteration 10
New best cost 20001.83 at ite

## _Construct your own Python program to implement the PJS heuristic for solving the TOP and test it in different instances._

Se ha implementado el algoritmo PJS para resolver TOP con las instancias p5.3.q.txt, p6.2.i.txt, p7.2.l.txt con diferentes alfas.

In [22]:
import math
import operator


def read_nodes(filename):
    nodes = []
    with open(filename) as instance:
        i = -3
        for line in instance:
            if i == -3:
                pass
            else:
                data = line.split(";")[1]
                if i == -2:
                    fleet_size = int(data)
                elif i == -1:
                    route_max_cost = float(data)
                else:
                    node_data = [float(x) for x in line.split(";")]
                    nodes.append(Node(i, node_data[0], node_data[1], node_data[2]))
            i += 1
    return nodes, route_max_cost, fleet_size


def pjs(alpha, nodes, max_cost, fleet_size):    
    def is_mergeable(i_node, j_node, i_route, j_route, ij_edge):
        return (
            i_route != j_route and
            i_node.finish_linked and j_node.start_linked and
            max_cost > i_route.cost + j_route.cost - ij_edge.savings
        )

    
    start_node, finish_node = nodes[0], nodes[-1]

    for node in nodes[1:-1]:
        sn_edge = Edge(start_node, node)
        nf_edge = Edge(node, finish_node)
        sn_edge.cost = math.sqrt((node.x - start_node.x)**2 + (node.y - start_node.y)**2)
        nf_edge.cost = math.sqrt((node.x - finish_node.x)**2 + (node.y - finish_node.y)**2)
        node.dn_edge = sn_edge
        node.nd_edge = nf_edge

    eff_list = []
    for i in range(1, len(nodes) - 2):
        i_node = nodes[i]
        for j in range(i+1, len(nodes) - 1):
            j_node = nodes[j]
            ij_edge = Edge(i_node, j_node)
            ji_edge = Edge(j_node, i_node)
            ij_edge.inverted_edge = ji_edge
            ji_edge.inverted_edge = ij_edge
            ij_edge.cost = math.sqrt((j_node.x - i_node.x)**2) + (j_node.y - i_node.y)**2
            ij_savings = i_node.nd_edge.cost + j_node.dn_edge.cost - ij_edge.cost
            e_reward = i_node.demand + j_node.demand
            ij_edge.savings = ij_savings
            ij_edge.efficiency = alpha * ij_savings + (1-alpha) * e_reward
            ji_savings = j_node.nd_edge.cost + i_node.dn_edge.cost - ji_edge.cost
            ji_edge.savings = ji_savings
            ji_edge.efficiency = alpha * ji_savings + (1-alpha) * e_reward
            eff_list += [ij_edge, ji_edge]

    eff_list.sort(key=operator.attrgetter("efficiency"), reverse=True)

    # construct solution
    solution = Solution()
    for node in nodes[1:-1]:
        sn_edge = node.dn_edge
        nf_edge = node.nd_edge
        snf_route = Route()
        snf_route.edges.append(sn_edge)
        snf_route.demand += node.demand
        snf_route.cost += sn_edge.cost
        snf_route.edges.append(nf_edge)
        snf_route.cost += nf_edge.cost
        node.in_route = snf_route
        node.start_linked = True
        node.finish_linked = True
        solution.routes.append(snf_route)
        solution.cost += snf_route.cost
        solution.demand += snf_route.demand

    while len(eff_list) > 0:
        index = 0
        ij_edge = eff_list.pop(index)
        i_node = ij_edge.origin
        j_node = ij_edge.end
        i_route = i_node.in_route
        j_route = j_node.in_route
        if is_mergeable(i_node, j_node, i_route, j_route, ij_edge):
            ji_edge = ij_edge.inverted_edge
            if ji_edge in eff_list:
                eff_list.remove(ji_edge)
            i_edge = i_route.edges[-1]
            i_route.edges.remove(i_edge)
            i_route.cost -= i_edge.cost
            i_node.finish_linked = False
            j_edge = j_route.edges[0]
            j_route.edges.remove(j_edge)
            j_route.cost -= j_edge.cost
            j_node.start_linked = False
            i_route.edges.append(ij_edge)
            i_route.cost += ij_edge.cost
            i_route.demand += j_node.demand
            j_node.in_route = i_route
            for edge in j_route.edges:
                i_route.edges.append(edge)
                i_route.cost += edge.cost
                i_route.demand += edge.end.demand
                edge.end.in_route = i_route
            solution.cost -= ij_edge.savings
            solution.routes.remove(j_route)

    solution.routes.sort(key=operator.attrgetter("demand"), reverse=True)
    for route in solution.routes[fleet_size:]:
        solution.demand -= route.demand
        solution.cost -= route.cost
        solution.routes.remove(route)

    return solution


alphas = [0.3, 0.5, 1.0]
files = ["p5.3.q.txt", "p6.2.i.txt", "p7.2.l.txt"]
for greed in alphas:
    for _file in files:
        instance_name = _file.split(".txt")[0]
        nodes, max_cost, fleet_size = read_nodes(_file)
        solution = pjs(greed, nodes, max_cost, fleet_size)
        print(f"\nReward obtained with PJS heuristics on {instance_name} is {solution.demand:.2f}")
        print(f"alpha: {greed} || Max cost: {max_cost} || Fleet size: {fleet_size}")
        for route in solution.routes:
            s = str(0)
            for edge in route.edges:
                s = s + '-' + str(edge.end.ID)
            print(f"Route: {s} || Reward = {route.demand:.2f} || Cost / Time = {route.cost:.2f}")



Reward obtained with PJS heuristics on p5.3.q is 1295.00
alpha: 0.3 || Max cost: 28.3 || Fleet size: 3
Route: 0-37-45-53-61-60-40-33-32-25-5-4-12-52-51-50-15-14-13-21-29-65 || Reward = 520.00 || Cost / Time = 25.61
Route: 0-46-54-62-59-48-41-24-17-6-3-11-27-22-20-28-65 || Reward = 395.00 || Cost / Time = 18.41
Route: 0-36-35-30-38-55-63-58-56-49-16-9-7-2-10-65 || Reward = 380.00 || Cost / Time = 23.24

Reward obtained with PJS heuristics on p6.2.i is 1074.00
alpha: 0.3 || Max cost: 27.5 || Fleet size: 2
Route: 0-57-54-53-49-48-43-41-37-34-29-27-42-36-35-28-21-15-10-6-4-63 || Reward = 582.00 || Cost / Time = 27.40
Route: 0-56-55-52-50-47-44-40-38-33-30-26-22-20-16-14-19-25-23-32-31-24-63 || Reward = 492.00 || Cost / Time = 27.05

Reward obtained with PJS heuristics on p7.2.l is 1183.00
alpha: 0.3 || Max cost: 120.0 || Fleet size: 2
Route: 0-85-67-62-97-92-83-71-68-42-20-19-96-91-86-57-49-47-44-10-8-56-48-46-12-11-7-6-4-2-1-94-80-73-53-37-36-35-32-13-60-43-31-22-14-21-18-89-84-59-54-50-

# _Personal Assessment_

En general los temas tratados hasta ahora me han resultado muy interesantes debido a su alta multidisciplinaridad y en la aplicación práctica de tecnologías y herramientas de uso relativamente moderno. Me parece muy interesante el concepto de simheurística, que unifique dos conceptos tan dispares como la simulación y las heurísticas y me atrae mucho el concepto de modelar el entorno teniendo en cuenta variables imprevisibles (caos).   
El nivel de dificultad me ha parecido medio, creo que requiere tener una buena base de matemáticas y programación pero es asequible. Por último, creo que la carga de trabajo ha sido un poco excesiva y algunas de las tareas no iban directamente relacionadas a los temas tratados sino que hablan de esta disciplina en general, como algunos papers.